In [19]:
names = [
    "/h.0/attn/Transpose",
    # "/h.0/attn/Mul",
]
names = [n + "_output_0" for n in names]
# names += ["/h.0/attn/Split_output_1"]

In [20]:
import onnx

model = onnx.load("models/tinyphysics.onnx")

value_info_protos = []
shape_info = onnx.shape_inference.infer_shapes(model)
for node in shape_info.graph.value_info:
    if node.name in names:
        value_info_protos.append(node)
model.graph.output.extend(value_info_protos)
onnx.checker.check_model(model, full_check=True)
onnx.save(model, "models/tmp_tinyphysics_output.onnx")

In [21]:
onnx.utils.extract_model("models/tmp_tinyphysics_output.onnx", "models/tmp_tinyphysics_extracted.onnx", ["states", "tokens"], names)